In [ ]:
pip install pyyaml h5py

In [24]:
import os
import cv2
import numpy as np
import pandas as pd
from urllib.parse import urlparse
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

In [19]:
def parse_url(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return f"{domain}.jpg"

def load_data():
    # loading and preprocessing  dataset 
    data = pd.read_csv("../dataset/annotations/annotations.csv")
    data = data[['id','image','score']]
        
    images = []
    for i in range(0,len(data)):
        filepath = data['image'][i]
        image = cv2.imread('../dataset/images/{0}'.format(filepath),cv2.IMREAD_COLOR)

        if image is not None:
            new_dimensions = (224, 224)
            resized_image = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)
            images.append(resized_image)

    return np.array(images), np.array(data['score'])

In [20]:
images, ratings = load_data()
images = images / 255.0  # normalize images

print(images.shape)

X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

print(y_train)


(107, 224, 224, 3)
[82. 29. 57. 57. 79. 54. 79. 57. 57. 79. 61. 57. 43. 64. 82. 57. 46. 61.
 79. 57. 57. 68. 75. 39. 57. 54. 82. 71. 79. 57. 75. 57. 71. 71. 79. 57.
 79. 57. 79. 54. 75. 61. 50. 75. 57. 79. 79. 57. 57. 36. 82. 79. 57. 57.
 64. 79. 79. 57. 79. 79. 79. 71. 79. 57. 71. 64. 57. 57. 57. 61. 57. 57.
 57. 82. 79. 79. 79. 57. 57. 79. 79. 75. 57. 57. 79.]


In [21]:
base_model = ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224,224,3)
    )

# Add custom top layers for regression
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # Output layer for regression

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 3854.5488 - mae: 60.9083 - val_loss: 2975.8613 - val_mae: 53.4763
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 730ms/step - loss: 2740.9727 - mae: 50.6863 - val_loss: 1943.9563 - val_mae: 42.7544
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 713ms/step - loss: 1755.6361 - mae: 40.0448 - val_loss: 1136.8898 - val_mae: 31.9405
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 674ms/step - loss: 1001.2656 - mae: 29.0079 - val_loss: 536.2789 - val_mae: 20.4413
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 686ms/step - loss: 540.8145 - mae: 19.9210 - val_loss: 196.9808 - val_mae: 10.3245
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 678ms/step - loss: 244.7569 - mae: 12.4653 - val_loss: 128.4268 - val_mae: 10.5189
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 659ms/step - loss: 222.7566 - mae: 12.1997 - val_loss: 223.7334 - val_mae: 12.7699
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 667ms/step - loss: 342.5424 - mae: 14.9450 - val_loss: 306.2551 - val_mae: 14.5130
Epoch 9/20
3/3 ━━━━━

In [22]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])
history_fine = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 197.8531 - mae: 10.9174 - val_loss: 121.2103 - val_mae: 10.4646
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 743ms/step - loss: 171.8457 - mae: 10.9327 - val_loss: 121.3298 - val_mae: 10.4846
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 706ms/step - loss: 178.0921 - mae: 11.1504 - val_loss: 121.6370 - val_mae: 10.5167
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 695ms/step - loss: 161.2450 - mae: 10.4227 - val_loss: 121.9569 - val_mae: 10.5463
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 711ms/step - loss: 182.8153 - mae: 11.1373 - val_loss: 122.2937 - val_mae: 10.5681
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 732ms/step - loss: 163.0367 - mae: 10.6092 - val_loss: 122.4759 - val_mae: 10.5820
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 730ms/step - loss: 159.5570 - mae: 10.7114 - val_loss: 122.7550 - val_mae: 10.5965
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 750ms/step - loss: 173.2232 - mae: 10.7565 - val_loss: 123.0404 - val_mae: 10.6078
Epoch 9/10
3/3 ━━━━━━━━━━━━

In [28]:
model.save("BetterSEOModel.h5")

In [33]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('BetterSEOModel.h5')

In [34]:
# Predict rating for a new screenshot
def predict_rating(screenshot):
    new_dimensions = (224,224)
    resized_image = cv2.resize(screenshot, new_dimensions, interpolation=cv2.INTER_AREA)
    screenshot = resized_image / 255.0
    screenshot = np.expand_dims(screenshot, axis=0)  
    predicted_rating = new_model.predict(screenshot)
    return predicted_rating[0][0]

# Load Screenshot and make prediction
new_screenshot = cv2.imread('../dataset/images/{0}'.format('architecturalrecord.png'),cv2.IMREAD_COLOR)
predicted_rating = predict_rating(new_screenshot)
print(f'Predicted Rating: {predicted_rating}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
Predicted Rating: 71.77581787109375
